In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import csv

In [45]:
# load vsech nasich + Honzovo dat o pozicich autobusu a spojeni do velke tabulky, odstaneni radku, kde je delay = NaN
data_nase = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW.csv', 
                         encoding='utf-8', low_memory=False)
data_Honza = pd.read_csv('out.c-cleaning.ALL_HONZA_NEW_WKEND.csv', 
                        encoding='utf-8', low_memory=False)
data_wkend = pd.read_csv ('out.c-cleaning.ALL_POSITIONS_NEW.csv', 
                         encoding='utf-8', low_memory=False)
 
data_wkend_nase = pd.read_csv ('out_clean_vikend_16_17.csv', 
                         encoding='utf-8', low_memory=False)

data = pd.concat([data_nase, data_Honza,data_wkend,data_wkend_nase], ignore_index = True)
data = data.dropna(axis=0,subset=['DELAY'],how='any')
data.head()
l=data['UNICORN'].unique()
len(l)

2161

In [46]:
# pridani sloupce s route_name
data['ROUTE_NAME'] = data['TRIP_ID'].str[:3]

In [47]:
# prevod sloupecku DATE na datovy format
data['DATE'] = data['DATE'] + " 2020"

In [48]:
data['DATE'] = pd.to_datetime(data['DATE'])

In [49]:
# vytvoreni noveho sloupce se dnem v tydnu
data['DAY_OF_WEEK'] = data['DATE'].dt.day_name()

In [50]:
# filtrace pouze linky 331 a 333, v ranni spicce, smer centrum -> novy mensi dataframe
ranni_331_333 = data.loc[(data['HEADSIGN'] == 'Praha,Kačerov') 
                     & (data['START_TIME'] < '12:00:00') 
                     & ((data['ROUTE_NAME'] == '333') | (data['ROUTE_NAME'] == '331'))
                     , ['UNICORN', 'TRIP_ID', 'START_TIME', 'HEADSIGN', 'DATE',
                        'DELAY', 'DELAY_STOP_ARRIVAL', 'DELAY_STOP_DEPARTURE',
                        'ORIGIN_TIMESTAMP', 'SHAPE_DIST_TRAVELED', 'SPEED', 'LATITUDE',
                        'LONGITUDE','ROUTE_NAME','DAY_OF_WEEK']]

# pocet unikatnich spoju
len(ranni_331_333['UNICORN'].unique().tolist())
# PRIDAT 331 - nechat HEADSIGN Kacerov a Budejovickou a START TIME -> ranni spicka

265

In [51]:
# pocet unikatnich trip_id v souboru 
len(ranni_331_333['TRIP_ID'].unique())

27

In [52]:
# DataFrame se vsemi jizdnimi rady (pro unikatnich 27 trip_id) -> vytvoreno v jizdni_rad.ipynb
info_stops_clean = pd.read_csv('jizdnirad.csv', encoding='utf-8', low_memory=False)

In [53]:
info_stops_clean.head()

,ARRIVAL_TIME,DEPARTURE_TIME,SHAPE_DIST_TRAVELED,STOP_ID,STOP_SEQUENCE,TRIP_ID,GEOMETRY_COORDINATES,STOP_NAME
0,9:31:00,9:31:00,0.00000,U2109Z2,1,331_243_200302,"[14.49348, 49.89506]","Jílové u Prahy,Náměstí"
1,9:33:00,9:33:00,0.65952,U2100Z2,2,331_243_200302,"[14.48552, 49.8974]","Jílové u Prahy,Učiliště"
2,9:37:00,9:37:00,2.37567,U1586Z2,3,331_243_200302,"[14.46857, 49.89977]","Jílové u Prahy,Kamenná Vrata"
3,9:39:00,9:39:00,4.03046,U1521Z2,4,331_243_200302,"[14.45916, 49.90944]","Okrouhlo,Zahořany"
4,9:43:00,9:43:00,5.86913,U1520Z2,5,331_243_200302,"[14.44766, 49.91885]",Okrouhlo


In [54]:
# vybrani sloupcu z JR
stop_dist = info_stops_clean.loc[:,['TRIP_ID','SHAPE_DIST_TRAVELED','STOP_ID','STOP_SEQUENCE','STOP_NAME']]
# stop_dist

In [55]:
# list unikatnich unicornu z df data (allpositions)
unicorn_list = ranni_331_333['UNICORN'].unique().tolist()
len(unicorn_list)

265

In [56]:
# prazdny novy DataFrame
final_tab = pd.DataFrame([], 
                         columns = ['DAY_OF_WEEK','TRIP_ID','DATE','UNICORN','STOP_ID','STOP_SEQUENCE','SHAPE_DIST_TRAVELED','DELAY'])
final_tab

,DAY_OF_WEEK,TRIP_ID,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,SHAPE_DIST_TRAVELED,DELAY


In [57]:
index = 0
final_tab = pd.DataFrame()

for uni in unicorn_list:
    #tohle je jen pro vypisování stavu procesu
    print(round(index/len(unicorn_list)*100), '%')
    index+=1
    
    #vyberu vsechny pozice od daného spoje
    unicorn_find = ranni_331_333[ranni_331_333['UNICORN'] == uni]
    
    #vyberu jen sloupce co mě zajímají
    bus_dist = unicorn_find.loc[:,['TRIP_ID','SHAPE_DIST_TRAVELED','DELAY','DATE','DAY_OF_WEEK']]
    
    #tady si vytáhnu opakující se trip_id i trip_date
    gtfs_trip_id = bus_dist['TRIP_ID'].tolist()[0]
    trip_date = bus_dist['DATE'].tolist()[0]
    trip_day = bus_dist['DAY_OF_WEEK'].tolist()[0]

    
    #pro každou zastávku v JŘ pro dané GTFS_TRIP_ID
    for i, stop in stop_dist[stop_dist['TRIP_ID'] == gtfs_trip_id].iterrows():
        
        #hledám nejbližší pozici tím, že si nastavím zatím nejbližší pozici jako hooodně daleko
        min_distance_so_far = 100000
        #i to zpoždění na nulu, ale prostě se vybere určitě alespoň jedna reálná
        stop_delay = 0
        
        for j, position in bus_dist.iterrows():
            
            # vypocita absolutni hodnotu rozdilu vzdalenosti
            actual_distance = abs(stop['SHAPE_DIST_TRAVELED'] - position['SHAPE_DIST_TRAVELED'])

            # a pokud je tahle pozice blíž než 100000 nebo blíž než poslední jiná nalezená
            if (min_distance_so_far > actual_distance):
                #tak si uližím aktuální nejbližší vzdálenost
                min_distance_so_far = actual_distance
                #a uložím si k ní i její zpoždění
                stop_delay = position['DELAY']

        # teď už mi to projelo k dané zastávce všema pozicema a snad jsem nalezl opravdu tu nejbližší
        #udělám si výsledný dataframe se všemi potřebnými údaji
        df = pd.DataFrame(
            {
                "TRIP_ID": [gtfs_trip_id], 
                "DAY_OF_WEEK": [trip_day],
                "DATE": [trip_date],
                "UNICORN": [uni],
                "STOP_ID": [stop['STOP_ID']],
                "STOP_SEQUENCE": [stop['STOP_SEQUENCE']],
                "STOP_NAME" : [stop['STOP_NAME']],
                "SHAPE_DIST_TRAVELED": [stop['SHAPE_DIST_TRAVELED']],
                "DELAY": [stop_delay],
            }
        )
        # a uložím do jednoho velkého
        final_tab = final_tab.append(df, ignore_index = True)

0 %
0 %
1 %
1 %
2 %
2 %
2 %
3 %
3 %
3 %
4 %
4 %
5 %
5 %
5 %
6 %
6 %
6 %
7 %
7 %
8 %
8 %
8 %
9 %
9 %
9 %
10 %
10 %
11 %
11 %
11 %
12 %
12 %
12 %
13 %
13 %
14 %
14 %
14 %
15 %
15 %
15 %
16 %
16 %
17 %
17 %
17 %
18 %
18 %
18 %
19 %
19 %
20 %
20 %
20 %
21 %
21 %
22 %
22 %
22 %
23 %
23 %
23 %
24 %
24 %
25 %
25 %
25 %
26 %
26 %
26 %
27 %
27 %
28 %
28 %
28 %
29 %
29 %
29 %
30 %
30 %
31 %
31 %
31 %
32 %
32 %
32 %
33 %
33 %
34 %
34 %
34 %
35 %
35 %
35 %
36 %
36 %
37 %
37 %
37 %
38 %
38 %
38 %
39 %
39 %
40 %
40 %
40 %
41 %
41 %
42 %
42 %
42 %
43 %
43 %
43 %
44 %
44 %
45 %
45 %
45 %
46 %
46 %
46 %
47 %
47 %
48 %
48 %
48 %
49 %
49 %
49 %
50 %
50 %
51 %
51 %
51 %
52 %
52 %
52 %
53 %
53 %
54 %
54 %
54 %
55 %
55 %
55 %
56 %
56 %
57 %
57 %
57 %
58 %
58 %
58 %
59 %
59 %
60 %
60 %
60 %
61 %
61 %
62 %
62 %
62 %
63 %
63 %
63 %
64 %
64 %
65 %
65 %
65 %
66 %
66 %
66 %
67 %
67 %
68 %
68 %
68 %
69 %
69 %
69 %
70 %
70 %
71 %
71 %
71 %
72 %
72 %
72 %
73 %
73 %
74 %
74 %
74 %
75 %
75 %
75 %
76 %
76 %
77 %
77 %
7

In [58]:
#tohle jsou všechny trip_id a podíváme se
len(final_tab['UNICORN'].unique())

265

In [59]:
final_tab['TRIP_ID'].unique()

array(['331_241_200302', '333_557_200302', '331_243_200302',
       '333_1064_200302', '333_715_200302', '333_1072_200302',
       '333_1052_200302', '331_260_200302', '333_677_200302',
       '333_751_200302', '333_1012_200302', '333_1094_200302',
       '333_1115_200302', '331_257_200302', '333_651_200307',
       '333_1026_200307', '333_653_200307', '333_671_200307',
       '333_1010_200307', '333_1037_200307', '333_636_200307',
       '333_1014_200307', '333_1047_200307', '333_655_200307',
       '333_1024_200307', '333_673_200307', '333_1012_200307'],
      dtype=object)

Unique trip_id --> sjednocení STOP_SEQUENCE ze zastávky Dolní Břežany,Náměstí


333_655_200307 >= 2
333_1012_200307>=14 
333_655_200307 Dolní Břežany,Náměstí
333_1047_200307(Zvole) >=7 
333_636_200307....Dolní Břežany,Náměstí
333_1037_200307(Březová-Oleško,Oleško)	>=14 
333_1010_200307(Březová-Oleško,Oleško) >= 19   
333_653_200307 ...Dolní Břežany,Náměstí	
333_1026_200307(zvole) >= 14
333_651_200307 . Dolní Břežany,Náměstí	
333_1115_200302 (zvOLE) >=12
333_1094_200302	(Březová-Oleško,Oleško) >= 14
333_1012_200302(Březová-Oleško,Oleško) >=14
333_751_200302 (Dolní Břežany,Obecní úřad) >= 2
333_677_200302 (Dolní Břežany,Obecní úřad) >= 2
333_1052_200302(zvole) >= 12
333_1072_200302(Zvole,Nová Zvole) >=14
-333_715_200302 (Dolní Břežany,Obecní úřad) >= 2
333_1064_200302 (Březová-Oleško,Oleško) >= 14
333_557_200302 (Dolní Břežany,Zálepy)>= 7

331_241_200302(Jílové u Prahy,Náměstí) >=14	
331_243_200302(Jílové u Prahy,Náměstí)>=14	
331_257_200302(Okrouhlo,Zahořany)>=11
331_260_200302(zvole)>=8

In [60]:
# jak vypadá trasa tohodle
# '333_677_200302_19_May' - trasa z Dolni Brezany, namesti -> chci z Pisnice
final_clean =   final_tab[(final_tab['TRIP_ID'] == '333_677_200302') & (final_tab['STOP_SEQUENCE'] >= 2) | 
(final_tab['TRIP_ID'] == '333_715_200302') & (final_tab['STOP_SEQUENCE'] >= 2)
| (final_tab['TRIP_ID'] == '333_751_200302') & (final_tab['STOP_SEQUENCE'] >= 2)
| (final_tab['TRIP_ID'] == '333_1012_200307') & (final_tab['STOP_SEQUENCE'] >= 14)
| (final_tab['TRIP_ID'] == '333_1047_200307') & (final_tab['STOP_SEQUENCE'] >= 7)
| (final_tab['TRIP_ID'] == '333_1037_200307') & (final_tab['STOP_SEQUENCE'] >= 14)
| (final_tab['TRIP_ID'] == '333_1010_200307') & (final_tab['STOP_SEQUENCE'] >= 19)
| (final_tab['TRIP_ID'] == '333_1026_200307') & (final_tab['STOP_SEQUENCE'] >= 14)
| (final_tab['TRIP_ID'] == '333_1115_200302') & (final_tab['STOP_SEQUENCE'] >= 12)
| (final_tab['TRIP_ID'] == '333_1094_200302') & (final_tab['STOP_SEQUENCE'] >= 14)
| (final_tab['TRIP_ID'] == '333_1012_200302') & (final_tab['STOP_SEQUENCE'] >= 14)
| (final_tab['TRIP_ID'] == '333_1052_200302') & (final_tab['STOP_SEQUENCE'] >= 12)
| (final_tab['TRIP_ID'] == '333_1072_200302') & (final_tab['STOP_SEQUENCE'] >= 14)
| (final_tab['TRIP_ID'] == '333_1064_200302') & (final_tab['STOP_SEQUENCE'] >= 14)
| (final_tab['TRIP_ID'] == '333_557_200302') & (final_tab['STOP_SEQUENCE'] >= 7)
| (final_tab['TRIP_ID'] == '331_241_200302') & (final_tab['STOP_SEQUENCE'] >= 14)
| (final_tab['TRIP_ID'] == '331_243_200302') & (final_tab['STOP_SEQUENCE'] >= 14)
| (final_tab['TRIP_ID'] == '331_257_200302') & (final_tab['STOP_SEQUENCE'] >= 11)
| (final_tab['TRIP_ID'] == '331_260_200302') & (final_tab['STOP_SEQUENCE'] >= 8)
| (final_tab['TRIP_ID'] == '333_636_200307')
| (final_tab['TRIP_ID'] == '333_655_200307')
| (final_tab['TRIP_ID'] == '333_653_200307')
| (final_tab['TRIP_ID'] == '333_651_200307')]


final_clean

# vyradit podle sequence stop prvni zastavky nezajimave pro  nas
# pridat sloupec delay diff -> nastavit delay diff na prvni zastavce na 0 a dalsi udelat rozdil jednotlivych delays

,TRIP_ID,DAY_OF_WEEK,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,STOP_NAME,SHAPE_DIST_TRAVELED,DELAY
13,331_241_200302,Wednesday,2020-05-06,331_241_200302_06_May,U1502Z2,14,"Dolní Břežany,Náměstí",13.17849,98.0
14,331_241_200302,Wednesday,2020-05-06,331_241_200302_06_May,U1501Z2,15,"Dolní Břežany,Na kopečku",13.80006,32.0
15,331_241_200302,Wednesday,2020-05-06,331_241_200302_06_May,U545Z2,16,Písnice,16.55734,45.0
16,331_241_200302,Wednesday,2020-05-06,331_241_200302_06_May,U546Z2,17,Ke Březině,16.93703,29.0
17,331_241_200302,Wednesday,2020-05-06,331_241_200302_06_May,U871Z2,18,Lipovická,17.13697,11.0
...,...,...,...,...,...,...,...,...,...
6181,331_257_200302,Wednesday,2020-05-13,331_257_200302_13_May,U749Z2,21,Tempo,15.98916,7.0
6182,331_257_200302,Wednesday,2020-05-13,331_257_200302_13_May,U656Z2,22,Sídliště Krč,16.32799,3.0
6183,331_257_200302,Wednesday,2020-05-13,331_257_200302_13_May,U893Z4,23,Zálesí,16.79198,12.0
6184,331_257_200302,Wednesday,2020-05-13,331_257_200302_13_May,U488Z7,24,Nemocnice Krč,17.34484,-21.0


In [69]:
#zkousim si jestli jsem usekla správný úsek
final_clean = final_tab[(final_tab['TRIP_ID'] == '331_260_200302') & (final_tab['STOP_SEQUENCE'] >= 8) ]
final_clean

,TRIP_ID,DAY_OF_WEEK,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,STOP_NAME,SHAPE_DIST_TRAVELED,DELAY
182,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U1502Z2,8,"Dolní Břežany,Náměstí",4.60988,-5.0
183,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U1501Z2,9,"Dolní Břežany,Na kopečku",5.23144,-8.0
184,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U545Z2,10,Písnice,7.98873,22.0
185,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U546Z2,11,Ke Březině,8.36842,-6.0
186,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U871Z2,12,Lipovická,8.56835,1.0
...,...,...,...,...,...,...,...,...,...
5851,331_260_200302,Tuesday,2020-05-12,331_260_200302_12_May,U749Z2,18,Tempo,11.45101,63.0
5852,331_260_200302,Tuesday,2020-05-12,331_260_200302_12_May,U656Z2,19,Sídliště Krč,11.78984,59.0
5853,331_260_200302,Tuesday,2020-05-12,331_260_200302_12_May,U893Z4,20,Zálesí,12.25383,29.0
5854,331_260_200302,Tuesday,2020-05-12,331_260_200302_12_May,U488Z7,21,Nemocnice Krč,12.80669,-1.0


In [65]:
final_tab[(final_tab['UNICORN'] == '331_260_200302_06_May')]

,TRIP_ID,DAY_OF_WEEK,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,STOP_NAME,SHAPE_DIST_TRAVELED,DELAY
175,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U1513Z2,1,Zvole,0.00000,23.0
176,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U6955Z2,2,"Ohrobec,Zvolská",0.75810,29.0
177,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U1512Z2,3,"Ohrobec,U Rybníka",1.23504,58.0
178,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U1506Z2,4,"Ohrobec,U Čisteckých",1.88757,1.0
179,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U1510Z2,5,"Dolní Břežany,Na Rovném",3.60474,11.0
180,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U1511Z2,6,"Dolní Břežany,Hřiště",4.00597,-61.0
181,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U1503Z2,7,"Dolní Břežany,Obecní úřad",4.45152,-2.0
182,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U1502Z2,8,"Dolní Břežany,Náměstí",4.60988,-5.0
183,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U1501Z2,9,"Dolní Břežany,Na kopečku",5.23144,-8.0
184,331_260_200302,Wednesday,2020-05-06,331_260_200302_06_May,U545Z2,10,Písnice,7.98873,22.0


In [35]:
# a jak tohodle
final_tab[final_tab['UNICORN']=='333_557_200302_27_Apr']

,TRIP_ID,DAY_OF_WEEK,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,STOP_NAME,SHAPE_DIST_TRAVELED,DELAY
2691,333_557_200302,Monday,2020-04-27,333_557_200302_27_Apr,U1509Z1,1,"Dolní Břežany,Zálepy",0.00000,165.0
2692,333_557_200302,Monday,2020-04-27,333_557_200302_27_Apr,U1508Z2,2,"Ohrobec,Károv",0.63032,-21.0
2693,333_557_200302,Monday,2020-04-27,333_557_200302_27_Apr,U1507Z2,3,Ohrobec,1.61071,102.0
2694,333_557_200302,Monday,2020-04-27,333_557_200302_27_Apr,U1506Z2,4,"Ohrobec,U Čisteckých",2.10890,62.0
2695,333_557_200302,Monday,2020-04-27,333_557_200302_27_Apr,U1504Z1,5,"Dolní Břežany,Lhota",3.35027,94.0
2696,333_557_200302,Monday,2020-04-27,333_557_200302_27_Apr,U1503Z2,6,"Dolní Břežany,Obecní úřad",5.37459,49.0
2697,333_557_200302,Monday,2020-04-27,333_557_200302_27_Apr,U1502Z2,7,"Dolní Břežany,Náměstí",5.53295,50.0
2698,333_557_200302,Monday,2020-04-27,333_557_200302_27_Apr,U1501Z2,8,"Dolní Břežany,Na kopečku",6.15451,106.0
2699,333_557_200302,Monday,2020-04-27,333_557_200302_27_Apr,U545Z2,9,Písnice,8.91179,69.0
2700,333_557_200302,Monday,2020-04-27,333_557_200302_27_Apr,U546Z2,10,Ke Březině,9.29149,57.0


In [ ]:
# no je vidět že první má 28 zastávek a druhý 16, ale ten první od zastávky 11 jede stejně jako ten druhý...
# úkol je to teď nějak zkonsolidovat...

In [33]:
final_tab[final_tab['TRIP_ID']=='333_1115_200302'][10:]

,TRIP_ID,DAY_OF_WEEK,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,STOP_NAME,SHAPE_DIST_TRAVELED,DELAY
103,333_1115_200302,Wednesday,2020-05-20,333_1115_200302_20_May,U1503Z2,11,"Dolní Břežany,Obecní úřad",8.51563,-43.0
104,333_1115_200302,Wednesday,2020-05-20,333_1115_200302_20_May,U1502Z2,12,"Dolní Břežany,Náměstí",8.67399,-15.0
105,333_1115_200302,Wednesday,2020-05-20,333_1115_200302_20_May,U1501Z2,13,"Dolní Břežany,Na kopečku",9.29555,-45.0
106,333_1115_200302,Wednesday,2020-05-20,333_1115_200302_20_May,U545Z2,14,Písnice,12.05284,-29.0
107,333_1115_200302,Wednesday,2020-05-20,333_1115_200302_20_May,U546Z2,15,Ke Březině,12.43253,-16.0
...,...,...,...,...,...,...,...,...,...
3357,333_1115_200302,Tuesday,2020-05-05,333_1115_200302_05_May,U749Z2,22,Tempo,15.51512,15.0
3358,333_1115_200302,Tuesday,2020-05-05,333_1115_200302_05_May,U656Z2,23,Sídliště Krč,15.85395,27.0
3359,333_1115_200302,Tuesday,2020-05-05,333_1115_200302_05_May,U893Z4,24,Zálesí,16.31794,50.0
3360,333_1115_200302,Tuesday,2020-05-05,333_1115_200302_05_May,U488Z7,25,Nemocnice Krč,16.87080,26.0


In [34]:
final_tab[final_tab['TRIP_ID']=='333_751_200302'][:]

,TRIP_ID,DAY_OF_WEEK,DATE,UNICORN,STOP_ID,STOP_SEQUENCE,STOP_NAME,SHAPE_DIST_TRAVELED,DELAY
217,333_751_200302,Wednesday,2020-05-20,333_751_200302_20_May,U1503Z2,1,"Dolní Břežany,Obecní úřad",0.00000,40.0
218,333_751_200302,Wednesday,2020-05-20,333_751_200302_20_May,U1502Z2,2,"Dolní Břežany,Náměstí",0.15836,39.0
219,333_751_200302,Wednesday,2020-05-20,333_751_200302_20_May,U1501Z2,3,"Dolní Břežany,Na kopečku",0.77992,-1.0
220,333_751_200302,Wednesday,2020-05-20,333_751_200302_20_May,U545Z2,4,Písnice,3.53721,-6.0
221,333_751_200302,Wednesday,2020-05-20,333_751_200302_20_May,U546Z2,5,Ke Březině,3.91690,-22.0
...,...,...,...,...,...,...,...,...,...
3331,333_751_200302,Monday,2020-05-04,333_751_200302_04_May,U749Z2,12,Tempo,6.99949,16.0
3332,333_751_200302,Monday,2020-05-04,333_751_200302_04_May,U656Z2,13,Sídliště Krč,7.33832,8.0
3333,333_751_200302,Monday,2020-05-04,333_751_200302_04_May,U893Z4,14,Zálesí,7.80231,12.0
3334,333_751_200302,Monday,2020-05-04,333_751_200302_04_May,U488Z7,15,Nemocnice Krč,8.35517,75.0


In [ ]:
#co třeba takto? :-)